# Optimization in Python

You might have noticed that we didn't do anything related to sparsity with scikit-learn models. A lot of the work we covered in the machine learning class is very recent research, and as such is typically not implemented by the popular libraries.

If we want to do things like sparse regression, we're going to have to roll up our sleeves and do it ourselves. For that, we need to be able to solve optimization problems. In Julia, we did this with JuMP. In Python, we'll use a similar library called *pyomo*.

# Installing pyomo

You can run the following command to install pyomo if you haven't already.

In [1]:
!pip install pyomo --user

# Intro to pyomo

Let's see how we translate a simple, 2 variable LP to pyomo code.

$$
\begin{align*}
\max_{x,y} \quad& x + 2y \\
\text{s.t.}\quad& x + y \leq 1 \\
& x, y \geq 0.
\end{align*}
$$

First thing is to import the pyomo functions:

In [2]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

Next, we construct a model object. This is a container for everything in our optimization problem: variables, constraints, solver options, etc.

In [3]:
m = ConcreteModel()

Next, we define the two decision variables in our optimization problem. We use the ``Var`` function to create the variables. The `within` keyword is used to specify the bounds on the variables, or equivalently the `bounds` keyword. The variables are added to the model object with names `x` and `y`.

In [4]:
m.x = Var(within=NonNegativeReals)
m.y = Var(bounds=(0, float('inf')))

We now add the single constraint of our problem using the ``Constraint`` function. We write it algebraically, and save the result to the model.

In [5]:
m.con = Constraint(expr=m.x + m.y <= 1)

We specify the objective function with the `Objective` function:

In [6]:
m.obj = Objective(sense=maximize, expr=m.x + 2 * m.y)

We solve the optimization problem by first specifying a solver using `SolverFactory` and then using this solver to solve the model:

In [7]:
solver = SolverFactory('gurobi')
solver.solve(m)

{'Problem': [{'Number of objectives': 1, 'Lower bound': 2.0, 'Name': 'x3', 'Number of variables': 3, 'Upper bound': 2.0, 'Number of continuous variables': 3, 'Sense': 'maximize', 'Number of constraints': 2, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of nonzeros': 3}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])], 'Solver': [{'Status': 'ok', 'Wall time': '0.000507831573486', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Time': 0.17572498321533203, 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Error rc': 0, 'Return code': '0'}]}

We can now inspect the solution values and optimal cost.

In [8]:
m.obj()

2.0

In [9]:
m.x.value

0.0

In [10]:
m.y.value

1.0

Let's put it all together to compare with Julia/JuMP

In [11]:
# Create model
m = ConcreteModel()
# Add variables
m.x = Var(within=NonNegativeReals)
m.y = Var(bounds=(0, float('inf')))
# Add constraint
m.con = Constraint(expr=m.x + m.y <= 1)
# Add objective
m.obj = Objective(sense=maximize, expr=m.x + 2 * m.y)
# Solve model
solver = SolverFactory('gurobi')
solver.solve(m)
# Inspect solution
print m.obj()
print m.x.value
print m.y.value

2.0
0.0
1.0


```julia
# Create model
m = Model(solver=GurobiSolver())
# Add variables
@variable(m, x >= 0)
@variable(m, y >= 0)
# Add constraint
@constraint(m, x + y <= 1)
# Add objective
@objective(m, Max, x + 2y)
# Solve model
solve(m)
# Inspect solution
@show getobjectivevalue(m)
@show getvalue(x)
@show getvalue(y)
```

### Exercise

Code and solve the following optimization problem:

$$
\begin{align*}
\min_{x,y} \quad& 3x - y \\
\text{s.t.}\quad& x + 2y \geq 1 \\
& x \geq 0 \\
& 0 \leq y \leq 1.
\end{align*}
$$

In [12]:
# Create the model
m = ConcreteModel()
# Add the variables
m.x = Var(within=NonNegativeReals)
m.y = Var(bounds=(0, 1))
# Add the constraint
m.con = Constraint(expr=m.x + 2 * m.y >= 1)
# Add the objective
m.obj = Objective(sense=minimize, expr=3 * m.x - m.y)

solver = SolverFactory('gurobi')
solver.solve(m)

print m.x.value, m.y.value

0.0 1.0


# Index sets

Let's now move to a more complicated problem. We'll look at a transportation problem:

$$
\begin{align}
\min & \sum\limits_{i = 1}^{m} \sum\limits_{j = 1}^{n} c_{ij} x_{ij}\\
& \sum\limits_{j = 1}^{n} x_{ij} \leq b_i && i = 1, \ldots, m\\
& \sum\limits_{i = 1}^{m} x_{ij} = d_j && j = 1, \ldots, n\\
& x_{ij} \ge 0 && i = 1, \ldots, m, j = 1, \ldots, n
\end{align}
$$

And with some data:

In [13]:
import numpy as np

m = 2  # Number of supply nodes
n = 5  # Number of demand nodes
# Supplies
b = np.array([1000, 4000])
# Demands
d = np.array([500, 900, 1800, 200, 700])
# Costs
c = np.array([[2, 4, 5, 2, 1], 
              [3, 1, 3, 2, 3]])

Now we can formulate the problem with pyomo

In [30]:
model = ConcreteModel()

First step is adding variables. We can add variables with indices by passing the relevant index sets to the `Var` constructor. In this case, we need a $m$-by$n$ matrix of variables:

In [31]:
model.x = Var(range(m), range(n), within=NonNegativeReals)

Now to add the constraints. We have to add one supply constraint for each factory, so we might try something like:

In [32]:
for i in range(m):
    model.supply = Constraint(expr=sum(model.x[i, j] for j in range(n)) <= b[i])

	This is usually indicative of a modelling error.
	To avoid this warning, use block.del_component() and block.add_component().


Can you see the problem? We are overwriting `model.supply` in each iteration of the loop, and so only the last constraint is applied.

Luckily, pyomo has a (not-so-easy) way to add multiple constraints at a time. We first define a *rule* that takes in the model and any required indices, and then returns the expression for the constraint:

In [33]:
def supply_rule(model, i):
    return sum(model.x[i, j] for j in range(n)) <= b[i]

We then add the constraint by referencing this rule along with the index set we want the constraint to be defined over:

In [34]:
model.supply2 = Constraint(range(m), rule=supply_rule)

We then apply the same approach for the demand constraints

In [35]:
def demand_rule(model, j):
    return sum(model.x[i, j] for i in range(m)) == d[j]
model.demand = Constraint(range(n), rule=demand_rule)

Finally, we add the objective:

In [41]:
model.obj = Objective(sense=minimize, 
    expr=sum(c[i, j] * model.x[i, j] 
    for i in range(m) for j in range(n)))

	This is usually indicative of a modelling error.
	To avoid this warning, use block.del_component() and block.add_component().


Now we can solve the problem

In [37]:
solver = SolverFactory('gurobi')
solver.solve(model)

{'Problem': [{'Number of objectives': 1, 'Lower bound': 8600.0, 'Name': 'x11', 'Number of variables': 11, 'Upper bound': 8600.0, 'Number of continuous variables': 11, 'Sense': 'minimize', 'Number of constraints': 9, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of nonzeros': 26}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])], 'Solver': [{'Status': 'ok', 'Wall time': '0.00135684013367', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Time': 0.05796098709106445, 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Error rc': 0, 'Return code': '0'}]}

It solved, so we can extract the results

In [39]:
flows = np.array([[model.x[i, j].value for j in range(n)] for i in range(m)])
flows

array([[  300.,     0.,     0.,     0.,   700.],
       [  200.,   900.,  1800.,   200.,     0.]])

We can also check the objective value for the cost of this flow

In [40]:
model.obj()

8600.0

For simplicity, here is the entire formulation and solving code together:

In [42]:
model = ConcreteModel()
# Variables
model.x = Var(range(m), range(n), within=NonNegativeReals)
# Supply constraint
def supply_rule(model, i):
    return sum(model.x[i, j] for j in range(n)) <= b[i]
model.supply2 = Constraint(range(m), rule=supply_rule)
# Demand constraint
def demand_rule(model, j):
    return sum(model.x[i, j] for i in range(m)) == d[j]
model.demand = Constraint(range(n), rule=demand_rule)
# Objective
model.obj = Objective(sense=minimize, 
    expr=sum(c[i, j] * model.x[i, j] 
    for i in range(m) for j in range(n)))
# Solve
solver = SolverFactory('gurobi')
solver.solve(model)
# Get results
flows = np.array([[model.x[i, j].value for j in range(n)] for i in range(m)])
model.obj()

8600.0

# Machine Learning

Now let's put our pyomo knowledge to use and implement some of the same methods we saw in the machine learning class

# Linear Regression

First let's just try a simple linear regression

In [13]:
def linear_regression(X, y):
    n, p = X.shape

    # Create model
    m = ConcreteModel()

    # Add variables
    m.beta = Var(range(p))

    # Add constraints

    # Add objective
    m.obj = Objective(sense=minimize, expr=sum(
        pow(y[i] - sum(X[i, j] * m.beta[j] for j in range(p)), 2) 
        for i in range(n)))

    solver = SolverFactory('gurobi')
    
    ## tee=True enables solver output
    # results = solver.solve(m, tee=True)
    results = solver.solve(m, tee=False)

    return [m.beta[j].value for j in range(p)]

Let's load up some data to test it out on:

In [45]:
from sklearn.datasets import load_boston
data = load_boston()
X = data.data
y = data.target

Try our linear regression function:

In [15]:
linear_regression(X, y)

[-0.0916297984025,
 0.0486751200558,
 -0.00377930013364,
 2.85636752114,
 -2.88077955069,
 5.92521437582,
 -0.00722447905437,
 -0.967995232723,
 0.170443393507,
 -0.00938925373266,
 -0.392425679598,
 0.0149832093403,
 -0.41697262342]

We can compare with sklearn to make sure it's right:

In [48]:
from sklearn.linear_model import LinearRegression
m = LinearRegression(fit_intercept=False)
m.fit(X, y)
m.coef_

/usr/local/lib/python2.7/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


array([ -9.16297843e-02,   4.86751203e-02,  -3.77930006e-03,
         2.85636751e+00,  -2.88077933e+00,   5.92521432e+00,
        -7.22447929e-03,  -9.67995240e-01,   1.70443393e-01,
        -9.38925373e-03,  -3.92425680e-01,   1.49832102e-02,
        -4.16972624e-01])

Just for reference, let's look back at how we do the same thing in JuMP!

```julia
using JuMP, Gurobi
function linear_regression(X, y)
    n, p = size(X)
    m = Model(solver=GurobiSolver())
    @variable(m, beta[1:p])
    @objective(m, Min, sum((y[i] - sum(X[i, j] * beta[j] for j = 1:p)) ^ 2 for i = 1:n))
    solve(m)
    getvalue(beta)
end
```

or even

```julia
using JuMP, Gurobi
function linear_regression(X, y)
    n, p = size(X)
    m = Model(solver=GurobiSolver())
    @variable(m, beta[1:p])
    @objective(m, Min, sum((y - X * beta) .^ 2))
    solve(m)
    getvalue(beta)
end
```

Much simpler!

### Exercise

Modify the linear regression formulation to include an intercept term, and compare to scikit-learn's LinearRegression with `fit_intercept=False` to make sure it's the same

In [46]:
def linear_regression_intercept(X, y):
    n, p = X.shape

    # Create model
    m = ConcreteModel()

    # Add variables
    m.beta = Var(range(p))
    m.b0 = Var()

    # Add constraints

    # Add objective
    m.obj = Objective(sense=minimize, expr=sum(
        pow(y[i] - sum(X[i, j] * m.beta[j] for j in range(p)) - m.b0, 2) 
        for i in range(n)))

    solver = SolverFactory('gurobi')
    
    ## tee=True enables solver output
    # results = solver.solve(m, tee=True)
    results = solver.solve(m, tee=False)

    return [m.beta[j].value for j in range(p)]
linear_regression_intercept(X, y)

[-0.107170537728,
 0.0463952198659,
 0.0208602390528,
 2.6885613979,
 -17.7957586652,
 3.8047524989,
 0.00075106019668,
 -1.47575881609,
 0.30565501599,
 -0.0123293463053,
 -0.953463554686,
 0.00939251301344,
 -0.525466631712]

In [49]:
m = LinearRegression(fit_intercept=True)
m.fit(X, y)
m.coef_

array([ -1.07170557e-01,   4.63952195e-02,   2.08602395e-02,
         2.68856140e+00,  -1.77957587e+01,   3.80475246e+00,
         7.51061703e-04,  -1.47575880e+00,   3.05655038e-01,
        -1.23293463e-02,  -9.53463555e-01,   9.39251272e-03,
        -5.25466633e-01])

# Robust Regression

We saw in the class that both ridge and lasso regression were robust versions of linear regression. Both of these are provided by `sklearn`, but we need to know how to implement them if we want to extend regression ourselves

In [17]:
def ridge_regression(X, y, rho):
    n, p = X.shape

    # Create model
    m = ConcreteModel()

    # Add variables
    m.beta = Var(range(p))

    # Add objective
    m.obj = Objective(sense=minimize, expr=sum(
        pow(y[i] - sum(X[i, j] * m.beta[j] for j in range(p)), 2) 
        for i in range(n)) + rho * sum(pow(m.beta[j], 2) for j in range(p)))

    solver = SolverFactory('gurobi')
    
    ## tee=True enables solver output
    # results = solver.solve(m, tee=True)
    results = solver.solve(m, tee=False)
    return [m.beta[j].value for j in range(p)]

In [18]:
ridge_regression(X, y, 100000)

[-0.0341059301407,
 0.0910768556555,
 -0.0207819760553,
 0.00195345979442,
 0.00107931312039,
 0.0365851040279,
 0.0311371812166,
 0.00813454917486,
 0.00037992962216,
 0.00124142522494,
 0.0251751186081,
 0.0536003278572,
 -0.0988725886313]

### Exercise

Implement Lasso regression

In [19]:
def lasso_regression(X, y, rho):
    n, p = X.shape

    # Create model
    m = ConcreteModel()

    # Add variables
    m.beta = Var(range(p))
    m.absb = Var(range(p))

    # Add constraints
    def absbeta1(m, j):
        return m.beta[j] <= m.absb[j]
    m.absb1 = Constraint(range(p), rule=absbeta1)
    def absbeta2(m, j):
        return -m.beta[j] <= m.absb[j]
    m.absb2 = Constraint(range(p), rule=absbeta2)
        

    # Add objective
    m.obj = Objective(sense=minimize, expr=sum(
        pow(y[i] - sum(X[i, j] * m.beta[j] for j in range(p)), 2) 
        for i in range(n)) + rho * sum(m.absb[j] for j in range(p)))

    solver = SolverFactory('gurobi')
    
    ## tee=True enables solver output
    # results = solver.solve(m, tee=True)
    results = solver.solve(m, tee=False)
    return [m.beta[j].value for j in range(p)]

In [20]:
lasso_regression(X, y, 1000)

[-0.0546999181104,
 0.0480414422887,
 6.94905241789e-15,
 1.03282248152e-14,
 1.68840591366e-15,
 4.12566721786,
 0.0313943077664,
 -0.278349914712,
 0.0914281435229,
 -0.00694827331105,
 -0.0895405878295,
 0.0160443216639,
 -0.557106933706]

# Sparse Regression

In [21]:
def sparse_regression(X, y, k):
    n, p = X.shape
    M = 1000

    # Create model
    m = ConcreteModel()

    # Add variables
    m.beta = Var(range(p))
    m.z = Var(range(p), within=Binary)

    # Add constraints
    def bigm1(m, j):
        return m.beta[j] <= M * m.z[j]
    m.bigm1 = Constraint(range(p), rule=bigm1)
    def bigm2(m, j):
        return m.beta[j] >= -M * m.z[j]
    m.bigm2 = Constraint(range(p), rule=bigm2)
        
    m.sparsity = Constraint(expr=sum(m.z[j] for j in range(p)) <= k)

    # Add objective
    m.obj = Objective(sense=minimize, expr=sum(
        pow(y[i] - sum(X[i, j] * m.beta[j] for j in range(p)), 2) 
        for i in range(n)))

    solver = SolverFactory('gurobi')
    
    ## tee=True enables solver output
    # results = solver.solve(m, tee=True)
    results = solver.solve(m, tee=False)
    return [m.beta[j].value for j in range(p)]

In [22]:
sparse_regression(X, y, 5)

[-0.206218898623,
 0.123866171721,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0145808602136,
 0.0,
 0.392458699757,
 0.0395267480155,
 0.0]

### Exercise

Try implementing the algorithmic framework for linear regression:
- sparsity constraints
- lasso regularization
- restrict highly correlated pairs of features
- nonlinear transformations (just $\sqrt(x)$ and $x^2$)

In [182]:
import numpy as np
from sklearn.preprocessing import normalize

def all_regression(X_orig, y, k, rho):
    n, p_orig = X_orig.shape
    M = 10
    
    X = np.concatenate(
        [X_orig, np.sqrt(X_orig), np.square(X_orig)], axis=1
    )
    p = X.shape[1]
    
    # Normalize data
    X = normalize(X, axis=0)
    y = (y - np.mean(y)) / np.linalg.norm(y)

    # Create model
    m = ConcreteModel()

    # Add variables
    m.beta = Var(range(p))
    m.z = Var(range(p), within=Binary)
    m.absb = Var(range(p))

    # Sparsity constraints
    def bigm1(m, j):
        return m.beta[j] <= M * m.z[j]
    m.bigm1 = Constraint(range(p), rule=bigm1)
    def bigm2(m, j):
        return m.beta[j] >= -M * m.z[j]
    m.bigm2 = Constraint(range(p), rule=bigm2)
    m.sparsity = Constraint(expr=sum(m.z[j] for j in range(p)) <= k)
    
    # Lasso constraints
    def absbeta1(m, j):
        return m.beta[j] <= m.absb[j]
    m.absb1 = Constraint(range(p), rule=absbeta1)
    def absbeta2(m, j):
        return -m.beta[j] <= m.absb[j]
    m.absb2 = Constraint(range(p), rule=absbeta2)
    
    # Correlation constraints
    corX = np.corrcoef(np.transpose(X))
    def cor_rule(m, i, j):
        if i > j and abs(corX[i, j]) > 0.8:
            return (sum(m.z[k] for k in range(i, p, p_orig)) + 
                    sum(m.z[k] for k in range(j, p, p_orig)) <= 1)
        else:
            return Constraint.Skip
    m.cor = Constraint(range(p_orig), range(p_orig), rule=cor_rule)
    
    # Nonlinear constraints
    def nl_rule(m, i):
        return sum(m.z[k] for k in range(i, p, p_orig)) <= 1
    m.nl = Constraint(range(p_orig), rule=nl_rule)

    # Add objective
    m.obj = Objective(sense=minimize, expr=sum(
        pow(y[i] - sum(X[i, j] * m.beta[j] for j in range(p)), 2) 
        for i in range(n)) + rho * sum(m.absb[j] for j in range(p)))

    solver = SolverFactory('gurobi')
    
    ## tee=True enables solver output
#     results = solver.solve(m, tee=True)
    results = solver.solve(m, tee=False)

    return np.array([m.beta[j].value for j in range(p)]).reshape(-1, p_orig)

In [183]:
all_regression(X, y, 6, 0)

array([[ 0.        ,  0.        ,  0.        ,  0.02823211,  0.        ,
         0.        ,  0.        , -0.08902008,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.29134352, -0.63119856],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.66037091,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.26103972,  0.        ,  0.        ]])

# Logistic Regression

Like JuMP, we need to use a new solver for the nonlinear problem. We can use Ipopt as before, except we have to set it up manually. You'll need to download Ipopt and add it to the PATH. 

On Mac, you can do this with Homebrew if you have it:

In [27]:
!brew install ipopt

The other way is to download a copy of ipopt and specify the path to it exactly when creating the solver. For example, I have a copy of Ipopt left over from JuMP, which I can use by modifying the SolverFactory line as indicated below:

In [189]:
def logistic_regression(X, y):
    n, p = X.shape
    
    # Convert y to (-1, +1)
    assert np.min(y) == 0
    assert np.max(y) == 1
    Y = y * 2 - 1
    assert np.min(Y) == -1
    assert np.max(Y) == 1

    # Create the model
    m = ConcreteModel()

    # Add variables
    m.b = Var(range(p))
    m.b0 = Var()

    # Set nonlinear objective function
    m.obj = Objective(sense=maximize, expr=-sum(
        log(1 + exp(-Y[i] * (sum(X[i, j] * m.b[j] for j in range(p)) + m.b0)))
        for i in range(n)))

    # Solve the model and get the optimal solutions
    solver = SolverFactory('ipopt')
    
    ## To use the version left over from Julia
    ## On Mac
#     solver = SolverFactory('ipopt', executable="~/.julia/v0.6/Homebrew/deps/usr/Cellar/ipopt/3.12.4_1/bin/ipopt")
    ## On Windows the path is probably under WinRPM
    # solver = SolverFactory('ipopt', executable='%HOME%\.julia\v0.6\WinRPM\...')
    
    solver.solve(m)
    return [m.b[j].value for j in range(p)], m.b0.value

Load up some data

In [190]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data.data
y = data.target

In [191]:
logistic_regression(X, y)

([4149.287451897711,
  -95.66578660735038,
  -122.72206478857777,
  -31.07341714231376,
  -30864.89323095169,
  36295.58728859856,
  -19052.223170338486,
  -19686.212584057354,
  14077.28373883507,
  -45229.08949784562,
  -2194.661967026749,
  98.5971333629637,
  810.8125884802604,
  -71.48236619171124,
  65261.00922698214,
  -62374.95392966877,
  52121.75890489646,
  -203728.94762214983,
  62535.54339961117,
  488515.82851625,
  -1086.5761316328774,
  -31.442974689271836,
  -43.12091429394959,
  7.303070144711609,
  3500.0884271415753,
  5108.102927332265,
  -4412.6083269231385,
  -4740.511086088049,
  -14164.862181354463,
  -35109.20858210006],
 -323.22751778394667)

### Exercise

Implement the regularized versions of logistic regression that scikit-learn provides:

![](http://scikit-learn.org/stable/_images/math/6a0bcf21baaeb0c2b879ab74fe333c0aab0d6ae6.png)

![](http://scikit-learn.org/stable/_images/math/760c999ccbc78b72d2a91186ba55ce37f0d2cf37.png)

In [194]:
def logistic_regression_l1(X, y, C):
    n, p = X.shape
    
    # Convert y to (-1, +1)
    assert np.min(y) == 0
    assert np.max(y) == 1
    Y = y * 2 - 1
    assert np.min(Y) == -1
    assert np.max(Y) == 1

    # Create the model
    m = ConcreteModel()

    # Add variables
    m.b = Var(range(p))
    m.b0 = Var()
    
    # Lasso constraints
    m.absb = Var(range(p))
    def absbeta1(m, j):
        return m.b[j] <= m.absb[j]
    m.absb1 = Constraint(range(p), rule=absbeta1)
    def absbeta2(m, j):
        return -m.b[j] <= m.absb[j]
    m.absb2 = Constraint(range(p), rule=absbeta2)
    
    # Set nonlinear objective function
    m.obj = Objective(sense=minimize, expr=sum(m.absb[j] for j in range(p)) + C * sum(
        log(1 + exp(-Y[i] * (sum(X[i, j] * m.b[j] for j in range(p)) + m.b0)))
        for i in range(n)))

    # Solve the model and get the optimal solutions
    solver = SolverFactory('ipopt')
    
    solver.solve(m)
    return [m.b[j].value for j in range(p)], m.b0.value

In [196]:
logistic_regression_l1(X, y, 100)

([4.395064126333529,
  -0.11536163425045783,
  -0.12337302600374518,
  -0.01936626591045223,
  -1.3388952674378297e-05,
  44.06235490000902,
  -5.36467540630947e-05,
  -114.59742041161189,
  3.1315376518085696,
  -1.954997632857952e-06,
  -0.0001529369038676667,
  1.613920185614702,
  1.1241880096170722,
  -0.2878801579365252,
  -1.3917240954019008e-05,
  59.48465136677453,
  9.709148208013277,
  -1.6874986336814742e-05,
  3.363939473126625e-05,
  3.4661698593929936e-06,
  -1.4250319711295396,
  -0.41697739115019217,
  -0.15308242915283365,
  0.0020392527812163955,
  -43.934285053178556,
  2.217865608175902,
  -10.250731059784785,
  -30.282661131054837,
  -14.863450800923257,
  -1.0088350216006994e-05],
 30.87167072501216)

In [197]:
def logistic_regression_l2(X, y, C):
    n, p = X.shape
    
    # Convert y to (-1, +1)
    assert np.min(y) == 0
    assert np.max(y) == 1
    Y = y * 2 - 1
    assert np.min(Y) == -1
    assert np.max(Y) == 1

    # Create the model
    m = ConcreteModel()

    # Add variables
    m.b = Var(range(p))
    m.b0 = Var()
    
    # Set nonlinear objective function
    m.obj = Objective(sense=minimize, expr=0.5 * sum(pow(m.b[j], 2) for j in range(p)) + C * sum(
        log(1 + exp(-Y[i] * (sum(X[i, j] * m.b[j] for j in range(p)) + m.b0)))
        for i in range(n)))

    # Solve the model and get the optimal solutions
    solver = SolverFactory('ipopt')
    
    solver.solve(m)
    return [m.b[j].value for j in range(p)], m.b0.value

In [200]:
logistic_regression_l2(X, y, 1000)

([2.8967817568863046,
  0.005420709146496229,
  -0.16647164395112038,
  -0.006703933468261533,
  -13.104363377099357,
  15.68101527364185,
  -17.59781077821828,
  -29.272302589577297,
  4.614842249435604,
  0.7355623311482853,
  -2.504921167755782,
  2.119095133372267,
  0.14716767197252492,
  -0.17226130026625208,
  -7.096244314033477,
  23.90707857998713,
  24.69263290413242,
  -6.048226654571491,
  8.913588143100506,
  4.7841488391800455,
  -1.3554658525312369,
  -0.46760590163349,
  -0.025470881419891362,
  -0.003363474811039968,
  -30.59427876947118,
  9.771808002605574,
  -8.807735054031562,
  -34.227909139073866,
  -14.698540645831052,
  -2.5928076418770276],
 34.314051046487776)